### Imports

In [ ]:
import earthaccess

import s3fs
import h5py

import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
from matplotlib import pyplot as plt
import cartopy.crs as ccrs

### Setup

In [ ]:
atl06_filename = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2023/12/27/ATL06_20231227235712_01402203_006_01.h5'
atl03_filename = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL03/006/2023/12/27/ATL03_20231227235712_01402203_006_02.h5'
gt = 'gt2l'


### Earthacess credentials

In [ ]:
auth = earthaccess.login()

credentials = earthaccess.get_s3_credentials("NSIDC")
print(credentials)

s3 = s3fs.S3FileSystem(key=credentials['accessKeyId'],
                       secret=credentials['secretAccessKey'],
                       token=credentials['sessionToken'])

# Open files
atl06 = h5py.File(s3.open(atl06_filename,'rb'),'r')
atl03 = h5py.File(s3.open(atl03_filename,'rb'),'r')


### Select an ATL06 segment

In [ ]:
# Here, we're selecting the segment with the highest n_fit_photons in the granule

n_fit_photons = atl06['/' + gt + '/land_ice_segments/fit_statistics/n_fit_photons'][:]

n_fit_photons_sort = sorted(n_fit_photons)

segment_id_selected = atl06['/' + gt + '/land_ice_segments/segment_id'][n_fit_photons == n_fit_photons_sort[-1]][0]

print('Selected ATL06 segment_id: {:d}'.format(segment_id_selected))


### Plot data from this segment from ATL03 and ATL06

In [ ]:
%matplotlib inline
plt.close()

# ATL06 segment_ids
segment_id_06 = atl06['/' + gt + '/land_ice_segments/segment_id'][:]

# ATL03 segment_ids
segment_ph_cnt = atl03['/' + gt + '/geolocation/segment_ph_cnt'][:]

h_ph_n = atl03['/' + gt + '/heights/h_ph'].shape[0]
segment_id = list(atl03['/' + gt + '/geolocation/segment_id'][:])
ph_index_beg = atl03['/' + gt + '/geolocation/ph_index_beg'][:]
segment_ph_cnt = atl03['/' + gt + '/geolocation/segment_ph_cnt'][:]

segment_id_ph = np.zeros(h_ph_n)
for igs in range(0,len(segment_id)):
    idx_gs = list(range(ph_index_beg[igs] - 1, \
                        ph_index_beg[igs] - 1 +\
                        segment_ph_cnt[igs]))
    segment_id_ph[idx_gs] = segment_id[igs]

# Select one segment
idx_06 = np.where( segment_id_06 == segment_id_selected )[0]

fig = plt.figure(figsize=(16,4))
gs = fig.add_gridspec(1, 5)
ax = list()
ax.append(fig.add_subplot(gs[0, 0], projection=ccrs.NorthPolarStereo(true_scale_latitude=70, central_longitude=-45)))
ax.append(fig.add_subplot(gs[0, 2:]))

# Plot all ATL06 locations from granule
latitude = atl06['/' + gt + '/land_ice_segments/latitude'][:]
longitude = atl06['/' + gt + '/land_ice_segments/longitude'][:]
ax[0].set_extent([-65, -25, 58, 88], ccrs.PlateCarree())
ax[0].plot(longitude, latitude, transform=ccrs.PlateCarree())
ax[0].plot(longitude[idx_06], latitude[idx_06], 'rx', transform=ccrs.PlateCarree())
ax[0].coastlines()
ax[0].gridlines()

# Plot ATL03
idx_03_ph = np.where( (segment_id_ph == segment_id_selected-1) )[0]
dist_ph_along = atl03['/' + gt + '/heights/dist_ph_along'][idx_03_ph]
h_ph = atl03['/' + gt + '/heights/h_ph'][idx_03_ph]
signal_conf_ph = atl03['/' + gt + '/heights/signal_conf_ph'][idx_03_ph,0]
ax[1].scatter(dist_ph_along-20., h_ph, s=2., c=signal_conf_ph, vmin=-1, vmax=5)
idx_03_ph = np.where( (segment_id_ph == segment_id_selected) )[0]
dist_ph_along = atl03['/' + gt + '/heights/dist_ph_along'][idx_03_ph]
h_ph = atl03['/' + gt + '/heights/h_ph'][idx_03_ph]
signal_conf_ph = atl03['/' + gt + '/heights/signal_conf_ph'][idx_03_ph,0]
c = ax[1].scatter(dist_ph_along, h_ph, s=2., c=signal_conf_ph, vmin=-1, vmax=5)
plt.colorbar(c, ax=ax[1], label='signal_ph_conf')

# Plot ATL06
x_atc = atl06['/' + gt + '/land_ice_segments/ground_track/x_atc'][idx_06]
h_mean = atl06['/' + gt + '/land_ice_segments/fit_statistics/h_mean'][idx_06]
dh_fit_dx = atl06['/' + gt + '/land_ice_segments/fit_statistics/dh_fit_dx'][idx_06]
w_surface_window_final = atl06['/' + gt + '/land_ice_segments/fit_statistics/w_surface_window_final'][idx_06]
ax[1].plot(0., h_mean, 'rx')
ax[1].plot([-20., 20.], (h_mean+w_surface_window_final/2) + [-20.*dh_fit_dx, +20.*dh_fit_dx], 'k--')
ax[1].plot([-20., 20.], (h_mean-w_surface_window_final/2) + [-20.*dh_fit_dx, +20.*dh_fit_dx], 'k--')

ax[1].set_xlabel('distance along track [m]')
ax[1].set_ylabel('height [m]')
ax[1].set_ylim( (h_mean-w_surface_window_final/2-2, h_mean+w_surface_window_final/2+2) )

print('{:d} photons in the ATL06 fit'.format(atl06['/' + gt + '/land_ice_segments/fit_statistics/n_fit_photons'][idx_06][0]))
